In [1]:
import pandas as pd

pre_separated_data = True  # Setar para False quando os dados de cadastro não estiverem separados em dataframes menores

## EDA

**Dados de treino (2022)**

Este é o conjunto de dados que você e sua equipe irão usar para trabalhar, criar o modelo, fazer testes e desenvolver a solução final. 

Transações: Data, PDV, Produto, Quantidade, Faturamento. 

Cadastro de produtos: Produto, Categoria, Descrição, + até 4 atributos.

Cadastro de PDVs: PDV, On/Off Prem, Categoria (c-store, g-store, liquor etc.), Zipcode. 

In [11]:
transacoes_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet"
cadastro_pdv_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet"
cadastro_prod_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet"

Separando o .parquet original do cadastro de produtos, para posteriormente acessá-lo via pandas:

In [3]:
if not pre_separated_data:
    import polars as pl

    df3 = pl.read_parquet(cadastro_prod_path)
    print(df3.shape)

    total_rows = df3.shape[0]
    chunk_size = total_rows // 10  # Integer division to get the chunk size

    for i in range(10):
        start_index = i * chunk_size
        end_index = start_index + chunk_size

        if i == 9:
            chunk = df3.slice(start_index)
        else:
            chunk = df3.slice(start_index, chunk_size)

        output_filename = (
            cadastro_prod_path.strip(".snappy.parquet") + f"_part_{i + 1}.parquet"
        )

        chunk.write_parquet(output_filename)
        print(f"Written {output_filename}")

In [50]:
df_transacoes = pd.read_parquet(transacoes_path)
df_cadastro_pdv = pd.read_parquet(cadastro_pdv_path)
df_cadastro_prod = pd.read_parquet(cadastro_prod_path)

In [51]:
#if pre_separated_data:
#    all_dfs = []
#    for i in range(10):
#        df3 = pd.read_parquet(
#            cadastro_prod_path.strip(".snappy.parquet") + f"_part_{i + 1}.parquet"
#        )
#        all_dfs.append(df3)
#    df_cadastro_prod = pd.concat(all_dfs, ignore_index=True)
#    del all_dfs

In [52]:
df_transacoes.shape, df_cadastro_pdv.shape, df_cadastro_prod.shape

((6560698, 11), (14419, 4), (7092, 8))

In [53]:
# Check the unique number of combinations of internal_store_id and internal_product_id on df_transacoes
df_transacoes[["internal_store_id", "internal_product_id"]].drop_duplicates().shape

(1044310, 2)

### **Identificando se há linhas duplicadas nos dataframes:**

In [54]:
df_cadastro_pdv.drop_duplicates(inplace=True)
df_cadastro_prod.drop_duplicates(inplace=True)
df_transacoes.drop_duplicates(inplace=True)

df_cadastro_pdv.shape, df_cadastro_prod.shape, df_transacoes.shape

((14419, 4), (7092, 8), (6560698, 11))

### **Identificando se há valores nulos / faltantes nos dataframes**

In [55]:
df_cadastro_pdv.isnull().sum()

pdv              0
premise          0
categoria_pdv    0
zipcode          0
dtype: int64

In [56]:
df_cadastro_prod.isnull().sum()  # Aparentemente a coluna label não invalida o uso desses dados, já é que ela especifica uma característica categória do produto (que pode ou não ser usada).

produto            0
categoria          0
descricao          0
tipos              0
label           1473
subcategoria      32
marca              0
fabricante         0
dtype: int64

In [57]:
df_cadastro_prod

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,2282334733936076502,Distilled Spirits,JOSEPH CARTRON CAFÉ LIQUEUR,Distilled Spirits,Core,Liqueurs & Cordials,Joseph Cartron Cafe,Spiribam
1,6091840953834683482,Distilled Spirits,SPRINGBANK 18 YEAR SINGLE MALT 700ML,Distilled Spirits,Specialty,Scotch Whisky,Springbank 18 Year Single Malt,Pacific Edge Wine & Spirits
2,1968645851245092408,Distilled Spirits,J BRANDT TRIPLE SEC 12/750ML 30PF,Distilled Spirits,Private Label,Liqueurs & Cordials,J Brandt Triple Sec,Sazerac Spirits
3,994706710729219179,Draft,REFORMATION CASHMERE IPA 1/4 KEG,Draft,In&Out,Other Draft,Reformation Cashmere Fresh Hop IPA,Reformation Brewery
4,9209550539540384349,Non-Alcohol,HELLA MOSCOW MULE 750ML,Non Alcohol,Core,Mixers,Hella Bitters Bloody Mary,Hella Bitter Llc
...,...,...,...,...,...,...,...,...
7087,2552936302329139135,Distilled Spirits,BOTANICAL GIN 1 LTR,Distilled Spirits,Close Out,Gin,Botanica Gin,Falcon Spirits
7088,5175418594860080319,Draft,CHIHUAHUA GUAVA LIME 1/6 KEG,Draft,Discontinued,Lager / Pilsner,Chihuahua Guava Lime,Chihuahua Brewing Company
7089,2514391530951310058,Distilled Spirits,TSMOORE PORT FINISH BOURBON 6/750ML 99PF,Distilled Spirits,Allocated,Bourbon Whiskey,Thomas S. Moore Chardonnay Cask Finish Bourbon,Sazerac Spirits
7090,9183313499769355186,Wine,RICH & CREAMY CHARDONNAY 2020,Wine < 14 %,Discontinued,White Wine,Rich & Creamy Chardonnay,Scotto Cellars


In [58]:
df_cadastro_prod.groupby("produto").value_counts()

produto              categoria          descricao                                                  tipos                  label         subcategoria     marca                                             fabricante               
1000423277513436210  Draft              REFORMATION A COLD ONE 97.1 PILSNER 1/4 KEG                Draft                  Core          Lager / Pilsner  Reformation A Cold One 97.1 Pilsner               Reformation Brewery          1
1000794189319388082  Draft              JEKYLL SOUR THANGS STRAW/LEM 1/6 KEG                       Draft                  In&Out        Sour / Wild Ale  Jekyll Sour Thangs Strawberry/Lemon               Jekyll Brewing               1
1000817480846862155  Package            BUD LIGHT SELTZER PEACH TEA 15/25 CN                       Package                Core          Other Package    Bud Light Seltzer Peach Tea                       AB Anheuser Busch Inc        1
1004943868572044494  Distilled Spirits  LA FAVORITE RHUM AGRICOLE BLA

In [59]:
#df_cadastro_prod[df_cadastro_prod.produto == "998529431391212014"].head(
#    10
#)  # Para um mesmo produto, há variações na descrição da subcategoria

In [60]:
#df_cadastro_prod.groupby("produto").size()

In [61]:
# Agrupando o dataframe pelo produto, checar quantas variações de subcategoria e categoria existem para cada produto
df_cadastro_prod.groupby("produto").agg(
    {
        "subcategoria": "nunique",
        "categoria": "nunique",
        "fabricante": "nunique",
        "label": "nunique",
    }
).reset_index().rename(
    columns={
        "subcategoria": "num_subcategorias",
        "categoria": "num_categorias",
        "fabricante": "num_fabricantes",
        "label": "num_labels",
    }
).sort_values(by=["num_subcategorias", "num_categorias"], ascending=False)

,produto,num_subcategorias,num_categorias,num_fabricantes,num_labels
0,1000423277513436210,1,1,1,1
1,1000794189319388082,1,1,1,1
2,1000817480846862155,1,1,1,1
3,1004943868572044494,1,1,1,1
4,1005164764874054592,1,1,1,1
...,...,...,...,...,...
6638,8766784970677530986,0,1,1,1
6654,8785300213971216422,0,1,1,0
6873,9069181404035127275,0,1,1,1
6972,9174279463424703825,0,1,1,1


In [62]:
df_transacoes.isnull().sum()

internal_store_id      0
internal_product_id    0
distributor_id         0
transaction_date       0
reference_date         0
quantity               0
gross_value            0
net_value              0
gross_profit           0
discount               0
taxes                  0
dtype: int64

In [63]:
df_transacoes.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758


`internal_store_id` de `df_transacoes` corresponde ao `pdv` da tabela `df_cadastro_pdv`. \
`internal_product_id` de `df_transacoes` corresponde ao `produto` da tabela `df_cadastro_prod`.

In [64]:
df_cadastro_pdv.head()

,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093


In [65]:
df_cadastro_prod.head()

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,2282334733936076502,Distilled Spirits,JOSEPH CARTRON CAFÉ LIQUEUR,Distilled Spirits,Core,Liqueurs & Cordials,Joseph Cartron Cafe,Spiribam
1,6091840953834683482,Distilled Spirits,SPRINGBANK 18 YEAR SINGLE MALT 700ML,Distilled Spirits,Specialty,Scotch Whisky,Springbank 18 Year Single Malt,Pacific Edge Wine & Spirits
2,1968645851245092408,Distilled Spirits,J BRANDT TRIPLE SEC 12/750ML 30PF,Distilled Spirits,Private Label,Liqueurs & Cordials,J Brandt Triple Sec,Sazerac Spirits
3,994706710729219179,Draft,REFORMATION CASHMERE IPA 1/4 KEG,Draft,In&Out,Other Draft,Reformation Cashmere Fresh Hop IPA,Reformation Brewery
4,9209550539540384349,Non-Alcohol,HELLA MOSCOW MULE 750ML,Non Alcohol,Core,Mixers,Hella Bitters Bloody Mary,Hella Bitter Llc


Arquivo de previsão no formato CSV ou Parquet, com as seguintes colunas:

| semana | pdv  | produto | quantidade |
|--------|------|---------|------------|
| 1      | 1023 | 123     | 120        |
| 2      | 1045 | 234     | 85         |
| 3      | 1023 | 456     | 110       |

No caso do csv, utilize “;” como caractere separador (exemplo: 1;1023;123;120) e encoding UTF-8.

**semana (número inteiro)**: número da semana (1 a 4 de janeiro/2023) \
**pdv (número inteiro)**: código do ponto de venda \
**produto (número inteiro)**: código do SKU \
**quantidade (número inteiro)**: previsão de vendas


## Organizar dados para obter Y

In [66]:
df_transacoes["mes"] = pd.to_datetime(df_transacoes["transaction_date"]).dt.month
df_transacoes["ano"] = pd.to_datetime(df_transacoes["transaction_date"]).dt.year

# Criar a coluna que define a semana em um dado mês do ano: semana 1 = dias 1-7, semana 2 = dias 8-14, semana 3 = dias 15-21, semana 4 = dias 22-28, semana 5 = dias 29-31
df_transacoes["semana"] = (
    (pd.to_datetime(df_transacoes["transaction_date"]).dt.day - 1) // 7
) + 1
#df_transacoes["semana"] = (
#    (pd.to_datetime(df_transacoes["transaction_date"]).dt.dayofyear - 1) // 7
#) + 1

In [67]:
df_transacoes.semana.value_counts()

semana
2    1765689
3    1471955
1    1418108
4    1408951
5     495995
Name: count, dtype: int64

In [68]:
df_transacoes.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes,mes,ano,semana
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375,7,2022,2
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000,3,2022,3
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000,9,2022,1
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000,9,2022,2
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758,2,2022,3


In [69]:
df_transacoes.value_counts(["semana", "transaction_date"]).sort_index().head(30)

semana  transaction_date
1       2022-01-01            170
        2022-01-02           1112
        2022-01-03           8833
        2022-01-04          20934
        2022-01-05          18039
        2022-01-06          17456
        2022-01-07          12957
        2022-02-01          22134
        2022-02-02          19963
        2022-02-03          28452
        2022-02-04          11448
        2022-02-05            606
        2022-02-06           1786
        2022-02-07          24453
        2022-03-01          23408
        2022-03-02          22547
        2022-03-03          19214
        2022-03-04          16378
        2022-03-05            202
        2022-03-06           3027
        2022-03-07          26414
        2022-04-01          17229
        2022-04-02            328
        2022-04-03           1754
        2022-04-04          30538
        2022-04-05          26130
        2022-04-06          24412
        2022-04-07          19417
        2022-05-01     

In [70]:
# Ao agrupar, manter as colunas mes e ano
df_transacoes_grouped = (
    df_transacoes.groupby(["semana","ano", "mes", "internal_store_id", "internal_product_id"]) # Excluí o distributor_id do agrupamento, assumindo que não será necessário em nossa análise
    .agg({"quantity": "sum",
    "gross_value": "sum",
    "net_value": "sum",
    "gross_profit": "sum",
    "discount": "sum",
    "taxes": "sum",
})
    .reset_index()
)
df_transacoes_grouped = df_transacoes_grouped.rename(
    columns={
        "internal_store_id": "pdv",
        "internal_product_id": "produto",
        "quantity": "quantidade",
    }
)

# Find rows that contain equal pdv and produto with same semana
(df_transacoes.groupby(["semana","mes", "ano", "internal_store_id", "internal_product_id"]).semana.value_counts() > 1).sum() # Cerca 278k casos em que, para um dado PDV+produto, foi vendido mais de uma vez na mesma semana

np.int64(278549)

In [71]:
df_transacoes_grouped.sort_values(by=["ano", "mes"])

,semana,ano,mes,pdv,produto,quantidade,gross_value,net_value,gross_profit,discount,taxes
0,1,2022,1,1001371918471115422,1029370090212151375,2.0,54.299999,49.740486,16.224486,0.000000,4.559516
1,1,2022,1,1001371918471115422,1120490062981954254,18.0,262.350006,234.992905,62.210903,53.099998,27.357101
2,1,2022,1,1001371918471115422,2239307647969388381,2.0,37.950001,31.870647,8.182648,5.100000,6.079354
3,1,2022,1,1001371918471115422,4353552881410365573,7.0,138.250000,118.302116,33.567116,2.800000,19.947886
4,1,2022,1,1001371918471115422,4797439216678436447,1.0,26.750000,24.470242,7.524241,0.000000,2.279758
...,...,...,...,...,...,...,...,...,...,...,...
6275222,5,2022,12,996297376394222790,9024979450937837276,1.0,24.600000,24.600000,8.560000,0.000000,0.000000
6275223,5,2022,12,997907800111849739,3516587220187345473,9.0,220.725006,218.615631,51.665630,24.750000,2.109375
6275224,5,2022,12,997907800111849739,4451855961536069898,5.0,164.250000,163.078125,46.283127,0.000000,1.171875
6275225,5,2022,12,997907800111849739,7029448080469668619,5.0,164.250000,163.078125,46.283127,0.000000,1.171875


In [72]:
df_transacoes

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes,mes,ano,semana
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375,7,2022,2
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000,3,2022,3
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000,9,2022,1
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000,9,2022,2
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758,2,2022,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560693,1439579822593789678,2523145330235258825,4,2022-12-06,2022-12-01,3.0,63.000000,61.379761,20.879761,0.000000,1.620240,12,2022,1
6560694,2357589266242432199,3148628429396585683,4,2022-05-20,2022-05-01,1.0,12.300000,12.300000,3.650000,2.800000,0.000000,5,2022,3
6560695,7245187752757348055,6668439675743950745,4,2022-08-17,2022-08-01,1.0,33.849998,31.570240,10.191240,0.000000,2.279758,8,2022,3
6560696,8952609383990245450,3894706280449257667,5,2022-02-03,2022-02-01,4.0,74.556000,73.655998,17.991999,15.600000,0.900000,2,2022,1


## Salvando o dataframe de cadastro filtrado pelo id do produto

In [43]:
#df_cadastro_prod = df_cadastro_prod.drop_duplicates(subset=["produto"])
#df_cadastro_prod.to_parquet(
#    "/data/challenges/hackathon_bigdata_2025/processed_data/cadastro_produtos.parquet",
#    index=False,
#)

## Lendo o novo dataframe de cadastro de produtos pré-processado e unindo-o com os outros:

In [44]:
#df_cadastro_prod = pd.read_parquet("/data/challenges/hackathon_bigdata_2025/processed_data/cadastro_produtos.parquet")

In [73]:
df_cadastro_pdv

,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093
...,...,...,...,...
14414,5047149578012031122,Off Premise,Package/Liquor,81501
14415,8352177618955815208,Off Premise,Package/Liquor,80219
14416,7754182004766664314,On Premise,Special Event,81004
14417,9044929558186581877,On Premise,Other On Premise,81226


In [74]:
rows_before_merge = df_transacoes_grouped.shape[0]
df_transacoes_grouped = df_transacoes_grouped.merge(
    df_cadastro_pdv, left_on="pdv", right_on="pdv", how="left",
)
rows_after_merge = df_transacoes_grouped.shape[0]
rows_before_merge, rows_after_merge, rows_before_merge - rows_after_merge # Cerca de 39k linhas são perdidas no merge ao usar "inner", indicando que há certos PDV_ids não presentes no dataframe de cadastro de PDVs.

(6275227, 6275227, 0)

In [75]:
df_transacoes_grouped

,semana,ano,mes,pdv,produto,quantidade,gross_value,net_value,gross_profit,discount,taxes,premise,categoria_pdv,zipcode
0,1,2022,1,1001371918471115422,1029370090212151375,2.0,54.299999,49.740486,16.224486,0.000000,4.559516,Off Premise,Convenience,30175.0
1,1,2022,1,1001371918471115422,1120490062981954254,18.0,262.350006,234.992905,62.210903,53.099998,27.357101,Off Premise,Convenience,30175.0
2,1,2022,1,1001371918471115422,2239307647969388381,2.0,37.950001,31.870647,8.182648,5.100000,6.079354,Off Premise,Convenience,30175.0
3,1,2022,1,1001371918471115422,4353552881410365573,7.0,138.250000,118.302116,33.567116,2.800000,19.947886,Off Premise,Convenience,30175.0
4,1,2022,1,1001371918471115422,4797439216678436447,1.0,26.750000,24.470242,7.524241,0.000000,2.279758,Off Premise,Convenience,30175.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6275222,5,2022,12,996297376394222790,9024979450937837276,1.0,24.600000,24.600000,8.560000,0.000000,0.000000,Off Premise,Convenience,30035.0
6275223,5,2022,12,997907800111849739,3516587220187345473,9.0,220.725006,218.615631,51.665630,24.750000,2.109375,Off Premise,Package/Liquor,80205.0
6275224,5,2022,12,997907800111849739,4451855961536069898,5.0,164.250000,163.078125,46.283127,0.000000,1.171875,Off Premise,Package/Liquor,80205.0
6275225,5,2022,12,997907800111849739,7029448080469668619,5.0,164.250000,163.078125,46.283127,0.000000,1.171875,Off Premise,Package/Liquor,80205.0


In [85]:
full_df = df_transacoes_grouped.merge(
    df_cadastro_prod, on="produto", how="left"
)

In [86]:
full_df.drop(columns=["internal_store_id", "internal_product_id"], errors="ignore", inplace=True)
full_df.head()

,semana,ano,mes,pdv,produto,quantidade,gross_value,net_value,gross_profit,discount,...,premise,categoria_pdv,zipcode,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,1,2022,1,1001371918471115422,1029370090212151375,2.0,54.299999,49.740486,16.224486,0.000000,...,Off Premise,Convenience,30175.0,Package,MICHELOB ULTRA 2/12/12 CN,Package,Core,Lager,Michelob Ultra,AB Anheuser Busch Inc
1,1,2022,1,1001371918471115422,1120490062981954254,18.0,262.350006,234.992905,62.210903,53.099998,...,Off Premise,Convenience,30175.0,Package,BUD LIGHT 2/12/8 CN,Package,Discontinued,Lager,Bud Light,AB Anheuser Busch Inc
2,1,2022,1,1001371918471115422,2239307647969388381,2.0,37.950001,31.870647,8.182648,5.100000,...,Off Premise,Convenience,30175.0,Package,NATURAL LIGHT 4/6/16 CN,Package,Core,Lager,Natural Light,AB Anheuser Busch Inc
3,1,2022,1,1001371918471115422,4353552881410365573,7.0,138.250000,118.302116,33.567116,2.800000,...,Off Premise,Convenience,30175.0,Package,NATURAL LIGHT 2/15/12 CN,Package,Core,Lager,Natural Light,AB Anheuser Busch Inc
4,1,2022,1,1001371918471115422,4797439216678436447,1.0,26.750000,24.470242,7.524241,0.000000,...,Off Premise,Convenience,30175.0,Package,BUD LIGHT LIME 2/12/12 CN,Package,Core,Lager,Bud Light Lime,AB Anheuser Busch Inc


In [87]:
cols = ['pdv', 'zipcode', 'produto', 'semana', 'mes', 'ano', 'categoria_pdv', 'categoria', 'subcategoria','premise', 'tipos', 'label', 'marca', 'fabricante', 'descricao',
       'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes', 'quantidade']
full_df = full_df[cols]
full_df.head()

,pdv,zipcode,produto,semana,mes,ano,categoria_pdv,categoria,subcategoria,premise,...,label,marca,fabricante,descricao,gross_value,net_value,gross_profit,discount,taxes,quantidade
0,1001371918471115422,30175.0,1029370090212151375,1,1,2022,Convenience,Package,Lager,Off Premise,...,Core,Michelob Ultra,AB Anheuser Busch Inc,MICHELOB ULTRA 2/12/12 CN,54.299999,49.740486,16.224486,0.000000,4.559516,2.0
1,1001371918471115422,30175.0,1120490062981954254,1,1,2022,Convenience,Package,Lager,Off Premise,...,Discontinued,Bud Light,AB Anheuser Busch Inc,BUD LIGHT 2/12/8 CN,262.350006,234.992905,62.210903,53.099998,27.357101,18.0
2,1001371918471115422,30175.0,2239307647969388381,1,1,2022,Convenience,Package,Lager,Off Premise,...,Core,Natural Light,AB Anheuser Busch Inc,NATURAL LIGHT 4/6/16 CN,37.950001,31.870647,8.182648,5.100000,6.079354,2.0
3,1001371918471115422,30175.0,4353552881410365573,1,1,2022,Convenience,Package,Lager,Off Premise,...,Core,Natural Light,AB Anheuser Busch Inc,NATURAL LIGHT 2/15/12 CN,138.250000,118.302116,33.567116,2.800000,19.947886,7.0
4,1001371918471115422,30175.0,4797439216678436447,1,1,2022,Convenience,Package,Lager,Off Premise,...,Core,Bud Light Lime,AB Anheuser Busch Inc,BUD LIGHT LIME 2/12/12 CN,26.750000,24.470242,7.524241,0.000000,2.279758,1.0


In [88]:
full_df.isna().sum()

pdv                   0
zipcode           39436
produto               0
semana                0
mes                   0
ano                   0
categoria_pdv     39436
categoria             0
subcategoria      10080
premise           39436
tipos                 0
label            521643
marca                 0
fabricante            0
descricao             0
gross_value           0
net_value             0
gross_profit          0
discount              0
taxes                 0
quantidade            0
dtype: int64

Temos agora um dataframe que engloba todos os metadados disponibilizados, com algumas ressalvas:

1. Há valores nulos na coluna label, subcategoria, categoria_pdv e zip_code; Referente às colunas com dados faltantes de PDV, é porque no dataframe de PDV tem alguns ids faltando.
2. Foi excluída a coluna distributor_id, assumindo-se que não será uma informação relevante (após se agrupar os dados por mes,semana,ano, e IDs).

In [89]:
full_df.to_parquet("/data/challenges/hackathon_bigdata_2025/processed_data/transacoes_merged.parquet")

In [90]:
full_df.shape

(6275227, 21)

## Criando um dataframe de targets (Y)

In [145]:
full_df = pd.read_parquet("/data/challenges/hackathon_bigdata_2025/processed_data/transacoes_merged.parquet")

In [98]:
full_df[['semana', 'pdv', 'produto', 'quantidade', 'mes', 'ano']]

,semana,pdv,produto,quantidade,mes,ano
0,1,1001371918471115422,1029370090212151375,2.0,1,2022
1,1,1001371918471115422,1120490062981954254,18.0,1,2022
2,1,1001371918471115422,2239307647969388381,2.0,1,2022
3,1,1001371918471115422,4353552881410365573,7.0,1,2022
4,1,1001371918471115422,4797439216678436447,1.0,1,2022
...,...,...,...,...,...,...
6275222,5,996297376394222790,9024979450937837276,1.0,12,2022
6275223,5,997907800111849739,3516587220187345473,9.0,12,2022
6275224,5,997907800111849739,4451855961536069898,5.0,12,2022
6275225,5,997907800111849739,7029448080469668619,5.0,12,2022


In [99]:
full_df[['semana', 'pdv', 'produto', 'quantidade', 'mes', 'ano']].duplicated().sum()

np.int64(0)

In [94]:
full_df[['semana', 'pdv', 'produto', 'quantidade', 'mes', 'ano']].to_parquet("/data/challenges/hackathon_bigdata_2025/processed_data/targets.parquet")

In [146]:
# Casos a serem filtrados (checar notebook eda_check_qtd_values)
print(f"Initial shape: {full_df.shape}")
filter_1 = (full_df.quantidade < 0) & ((full_df.quantidade / full_df.quantidade.astype(int)) != 1)
full_df = full_df[~filter_1]
print(f"Shape after 1st filter: {full_df.shape}")

filter_2 = (full_df.quantidade < 0) & ((full_df.gross_value >= 0) | (full_df.net_value >= 0) | (full_df.gross_profit >= 0) | (full_df.discount <=0))
full_df = full_df[~filter_2]

print(f"Shape after 2nd filter: {full_df.shape}")

filter_3 = (full_df.quantidade == 0) & ((full_df.gross_value !=0) | (full_df.net_value != 0) | (full_df.gross_profit !=0) | (full_df.taxes != 0) | (full_df.discount <= 0))
full_df = full_df[~filter_3]
print(f"Shape after 3rd filter: {full_df.shape}")


Initial shape: (6275227, 21)
Shape after 1st filter: (6255828, 21)
Shape after 2nd filter: (6221017, 21)
Shape after 3rd filter: (6166033, 21)


In [147]:
df = full_df.copy()

# Criamos uma cópia para representar o "mês seguinte"
df_next = df[['pdv', 'produto', 'mes', 'semana', 'ano', 'quantidade']].copy()
df_next['mes'] = df_next['mes'] - 1      # volta 1 mês p/ alinhar com o original
df_next.rename(columns={'quantidade': 'target'}, inplace=True)

# Faz o merge: para cada linha em df, tenta achar pdv/produto/mes na df_next
result = df.merge(
    df_next,
    how='left',
    on=['pdv', 'produto', 'mes', 'semana', 'ano']
)

# Substitui NaN por "Not available"
result['target'] = result['target'].fillna('Not available')

In [149]:
result.target.value_counts()

target
 Not available    4248304
 1.0               704768
 2.0               418908
 3.0               206577
 4.0               124653
                   ...   
 392.0                  1
 24.75                  1
 6.583333               1
 211.0                  1
-7.0                    1
Name: count, Length: 712, dtype: int64

In [150]:
(result[result.mes == 9].target != "Not Available").sum(), (result[result.mes == 10].target != "Not Available").sum()

(np.int64(792954), np.int64(508718))

In [151]:
(result[result.mes == 11].target != "Not Available").sum()

np.int64(523157)

In [152]:
result[result.mes == 12].target.value_counts()

target
Not available    518066
Name: count, dtype: int64

In [153]:
result.to_pickle("/data/challenges/hackathon_bigdata_2025/processed_data/merged_transactions_with_targets.pkl")